Set your drive path here

In [8]:
drive_path = '/media/stephen/Brain20191/allen-brain-observatory/visual-coding-2p/'

In [9]:
import os
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
from utils.data_wrappers import *
manifest_file = os.path.join(drive_path,'manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)


Print the various values for brain area, layer, celltype

In [3]:
brain_areas = boc.get_all_targeted_structures()
brain_areas

['VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISrl']

In [4]:
depths = boc.get_all_imaging_depths()
depths

[175,
 185,
 195,
 200,
 205,
 225,
 250,
 265,
 275,
 276,
 285,
 300,
 320,
 325,
 335,
 350,
 365,
 375,
 390,
 400,
 550,
 570,
 625]

In [5]:
cell_types = boc.get_all_cre_lines()
cell_types

['Cux2-CreERT2',
 'Emx1-IRES-Cre',
 'Fezf2-CreER',
 'Nr5a1-Cre',
 'Ntsr1-Cre_GN220',
 'Pvalb-IRES-Cre',
 'Rbp4-Cre_KL100',
 'Rorb-IRES2-Cre',
 'Scnn1a-Tg3-Cre',
 'Slc17a7-IRES2-Cre',
 'Sst-IRES-Cre',
 'Tlx3-Cre_PL56',
 'Vip-IRES-Cre']

Experiment counts by brain area, layer, cell type

In [6]:
exps = boc.get_ophys_experiments(stimuli=[stim_info.DRIFTING_GRATINGS])
exps_df = pd.DataFrame.from_dict(exps)
exps_df.groupby(['targeted_structure', 'imaging_depth', 'cre_line'])['id'].count()

targeted_structure  imaging_depth  cre_line         
VISal               175            Cux2-CreERT2         6
                                   Emx1-IRES-Cre        2
                                   Slc17a7-IRES2-Cre    1
                    265            Emx1-IRES-Cre        1
                    275            Cux2-CreERT2         6
                                   Emx1-IRES-Cre        1
                                   Rorb-IRES2-Cre       6
                                   Slc17a7-IRES2-Cre    1
                                   Sst-IRES-Cre         1
                    276            Cux2-CreERT2         1
                    300            Nr5a1-Cre            1
                    350            Nr5a1-Cre            5
                    365            Emx1-IRES-Cre        1
                    375            Emx1-IRES-Cre        2
                                   Rbp4-Cre_KL100       6
VISam               175            Cux2-CreERT2         5
                   

Get cells given brain area, depth, cell type

In [7]:
"""
Get cells that match the given selectors.
"""
def get_cells(brain_area, depth, cell_type):
    cells = boc.get_cell_specimens()
    cells = pd.DataFrame.from_records(cells)

    exps = boc.get_ophys_experiments(stimuli=[stim_info.DRIFTING_GRATINGS],
        targeted_structures = [brain_area],
        imaging_depths = [depth],
        cre_lines = [cell_type])
    # Can't find [targeted_structure, cre_line] in cell table, so have to combine w/ exps.
    exps_df = pd.DataFrame.from_dict(exps)
    if len(exps_df)== 0:
        return []
    is_in_relevant_container_mask = False * len(cells)
    for relevant_container_id in (exps_df.experiment_container_id.values):
        is_in_relevant_container_mask |= (cells.experiment_container_id == relevant_container_id)
    return cells[is_in_relevant_container_mask]
#cells_df = get_cells('VISl', 175, 'Ai148(TIT2L-GC6f-ICL-tTA2)')
#cells_df


Get cell counts per brain area, depth, cell type

In [8]:
exps = boc.get_ophys_experiments(stimuli=[stim_info.DRIFTING_GRATINGS])
exps_df = pd.DataFrame.from_dict(exps)
for brain_area in brain_areas:
    for depth in depths:
        for cell_type in boc.get_all_cre_lines():
            if len(exps_df[
                (exps_df['targeted_structure'] == brain_area) &
                (exps_df['imaging_depth'] == depth) &
                (exps_df['cre_line'] == cell_type)
                ]) == 0:
                continue
            cell_count = len(get_cells(brain_area, depth, cell_type))
            if cell_count == 0:
                continue
            print (brain_area, depth, cell_type, cell_count)

VISal 175 Cux2-CreERT2 1226
VISal 175 Emx1-IRES-Cre 512
VISal 175 Slc17a7-IRES2-Cre 64
VISal 265 Emx1-IRES-Cre 346
VISal 275 Cux2-CreERT2 1665
VISal 275 Emx1-IRES-Cre 492
VISal 275 Rorb-IRES2-Cre 1242
VISal 275 Slc17a7-IRES2-Cre 310
VISal 275 Sst-IRES-Cre 24
VISal 276 Cux2-CreERT2 212
VISal 300 Nr5a1-Cre 78
VISal 350 Nr5a1-Cre 142
VISal 365 Emx1-IRES-Cre 162
VISal 375 Emx1-IRES-Cre 275
VISal 375 Rbp4-Cre_KL100 441
VISam 175 Cux2-CreERT2 619
VISam 175 Emx1-IRES-Cre 167
VISam 175 Slc17a7-IRES2-Cre 235
VISam 275 Cux2-CreERT2 997
VISam 275 Emx1-IRES-Cre 195
VISam 275 Rorb-IRES2-Cre 735
VISam 320 Nr5a1-Cre 5
VISam 325 Nr5a1-Cre 75
VISam 350 Nr5a1-Cre 91
VISam 375 Emx1-IRES-Cre 95
VISam 375 Rbp4-Cre_KL100 355
VISl 175 Cux2-CreERT2 1208
VISl 175 Emx1-IRES-Cre 1122
VISl 175 Slc17a7-IRES2-Cre 1115
VISl 175 Vip-IRES-Cre 161
VISl 250 Sst-IRES-Cre 56
VISl 250 Vip-IRES-Cre 27
VISl 275 Cux2-CreERT2 1584
VISl 275 Emx1-IRES-Cre 976
VISl 275 Pvalb-IRES-Cre 139
VISl 275 Rorb-IRES2-Cre 1191
VISl 275 Slc1

Load filters

In [11]:
dg_response_filter = get_dg_response_filter_from_saskia()
dg_response_filter

NameError: name 'get_dg_response_filter_from_saskia' is not defined

Show cell counts before and after filtering.
See https://pastebin.com/NGzvE0Ky for result